In [1]:
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import pickle
import warnings
warnings.filterwarnings('ignore')

In [3]:
print("Predictor de partidos VCT")
print("="*60)

# Cargamos el modelo
print("\nCargando el modelo...")

with open('../models/saved_models/best_model_lr.pkl', 'rb') as f:
    model = pickle.load(f)

with open('../models/saved_models/scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('../models/saved_models/features.pkl', 'rb') as f:
    features = pickle.load(f)

print("Modelo cargado: Logistic Regression")
print(f"Features: {len(features)}")

# Cargamos los datos historicos
df = pd.read_parquet('../data/processed/model_ready_dataset.parquet')
print(f"Datos históricos: {len(df):,} partidos")

Predictor de partidos VCT

Cargando el modelo...
Modelo cargado: Logistic Regression
Features: 30
Datos históricos: 8,617 partidos


In [4]:
# Calculamos stats históricas promedio de cada equipo

print("Calculando stat promedio por equipo")
print("="*60)

# Stats cuando el equipo jugó como Team A
stats_as_teamA = df.groupby('teamA_name').agg({
    'teamA_kills': 'mean',
    'teamA_deaths': 'mean',
    'teamA_assists': 'mean',
    'teamA_acs': 'mean',
    'teamA_rating': 'mean',
    'teamA_first_kills': 'mean',
    'teamA_first_deaths': 'mean',
    'teamA_kd': 'mean',
    'teamA_ka': 'mean',
    'teamA_fkfd_diff': 'mean'
}).reset_index().rename(columns={'teamA_name': 'equipo'})

# Renombrar columnas para que sean genéricas
stats_as_teamA.columns = ['equipo', 'kills', 'deaths', 'assists', 'acs', 
                           'rating', 'first_kills', 'first_deaths', 
                           'kd', 'ka', 'fkfd_diff']

# Stats cuando el equipo jugó como Team B
stats_as_teamB = df.groupby('teamB_name').agg({
    'teamB_kills': 'mean',
    'teamB_deaths': 'mean',
    'teamB_assists': 'mean',
    'teamB_acs': 'mean',
    'teamB_rating': 'mean',
    'teamB_first_kills': 'mean',
    'teamB_first_deaths': 'mean',
    'teamB_kd': 'mean',
    'teamB_ka': 'mean',
    'teamB_fkfd_diff': 'mean'
}).reset_index().rename(columns={'teamB_name': 'equipo'})

stats_as_teamB.columns = ['equipo', 'kills', 'deaths', 'assists', 'acs',
                           'rating', 'first_kills', 'first_deaths',
                           'kd', 'ka', 'fkfd_diff']

# Combinar ambas (promedio de cuando jugó como A y como B)
all_stats = pd.concat([stats_as_teamA, stats_as_teamB], ignore_index=True)

team_avg_stats = all_stats.groupby('equipo').mean().reset_index()
print(f"\nStats calculadas para {len(team_avg_stats):,} equipos")
print(f"\nEjemplo - Stats de 5 equipos:")
display(team_avg_stats.head())

# Guardar lista de equipos disponibles
available_teams = sorted(team_avg_stats['equipo'].unique())
print(f"\nEquipos disponibles: {len(available_teams)}")

Calculando stat promedio por equipo

Stats calculadas para 3,550 equipos

Ejemplo - Stats de 5 equipos:


,equipo,kills,deaths,assists,acs,rating,first_kills,first_deaths,kd,ka,fkfd_diff
0,#1 Victory Royale,166.0,177.0,90.0,211.2,0.974,25.0,22.0,0.937853,1.446328,3.0
1,#FIXBUCKY,189.0,231.0,57.0,181.8,NaN,27.0,33.0,0.818182,1.064935,-6.0
2,#PROPLAYERGAGAL,132.0,155.0,45.0,197.2,NaN,18.0,22.0,0.851613,1.141935,-4.0
3,+W party,178.5,210.0,64.5,185.9,NaN,25.0,30.5,0.824301,1.111888,-5.5
4,010 Esports,140.0,156.0,39.0,187.2,0.922,21.0,21.0,0.897436,1.147436,0.0



Equipos disponibles: 3550


In [5]:
# Función para predecir resultado entre 2 equipos

def predict_match(team_a, team_b):
    # Verificar que ambos equipos existan
    if team_a not in available_teams:
        return {"error": f" '{team_a}' no encontrado en la base de datos"}
    if team_b not in available_teams:
        return {"error": f" '{team_b}' no encontrado en la base de datos"}
    
    # Obtener stats de cada equipo
    stats_a = team_avg_stats[team_avg_stats['equipo'] == team_a].iloc[0]
    stats_b = team_avg_stats[team_avg_stats['equipo'] == team_b].iloc[0]

    # Construir las features en el orden correcto
    match_features = {}
    
    # Features de Team A
    match_features['teamA_kills'] = stats_a['kills']
    match_features['teamA_deaths'] = stats_a['deaths']
    match_features['teamA_assists'] = stats_a['assists']
    match_features['teamA_acs'] = stats_a['acs']
    match_features['teamA_rating'] = stats_a['rating']
    match_features['teamA_first_kills'] = stats_a['first_kills']
    match_features['teamA_first_deaths'] = stats_a['first_deaths']
    match_features['teamA_kd'] = stats_a['kd']
    match_features['teamA_ka'] = stats_a['ka']
    match_features['teamA_fkfd_diff'] = stats_a['fkfd_diff']
    
    # Features de Team B
    match_features['teamB_kills'] = stats_b['kills']
    match_features['teamB_deaths'] = stats_b['deaths']
    match_features['teamB_assists'] = stats_b['assists']
    match_features['teamB_acs'] = stats_b['acs']
    match_features['teamB_rating'] = stats_b['rating']
    match_features['teamB_first_kills'] = stats_b['first_kills']
    match_features['teamB_first_deaths'] = stats_b['first_deaths']
    match_features['teamB_kd'] = stats_b['kd']
    match_features['teamB_ka'] = stats_b['ka']
    match_features['teamB_fkfd_diff'] = stats_b['fkfd_diff']
    
    # Features diferenciales
    match_features['diff_kills'] = stats_a['kills'] - stats_b['kills']
    match_features['diff_deaths'] = stats_a['deaths'] - stats_b['deaths']
    match_features['diff_assists'] = stats_a['assists'] - stats_b['assists']
    match_features['diff_acs'] = stats_a['acs'] - stats_b['acs']
    match_features['diff_rating'] = stats_a['rating'] - stats_b['rating']
    match_features['diff_first_kills'] = stats_a['first_kills'] - stats_b['first_kills']
    match_features['diff_first_deaths'] = stats_a['first_deaths'] - stats_b['first_deaths']
    match_features['diff_kd'] = stats_a['kd'] - stats_b['kd']
    match_features['diff_ka'] = stats_a['ka'] - stats_b['ka']
    match_features['diff_fkfd_diff'] = stats_a['fkfd_diff'] - stats_b['fkfd_diff']

    # Convertir a DataFrame en el orden de las features originales
    X_match = pd.DataFrame([match_features])
    X_match = X_match[features]

    # Rellenar Nan con mediana 
    X_match = X_match.fillna(X_match.median())

    # Escalar
    X_match_scaled = scaler.transform(X_match)

    # Predecir 
    prediction = model.predict(X_match_scaled)[0]
    probabilities = model.predict_proba(X_match_scaled)[0]

    # Resultado
    ganador = team_a if prediction == 1 else team_b
    prob_a = probabilities[1] * 100
    prob_b = probabilities[0] * 100

    return {
        "team_a": team_a,
        "team_b": team_b,
        "ganador_predicho": ganador,
        "probabilidad_team_a": prob_a,
        "probabilidad_team_b": prob_b,
        "confianza": max(prob_a, prob_b)
    }



In [ ]:
from ipywidgets import interact, Dropdown, Button, Output, VBox, HBox, HTML
from IPython.display import display, clear_output

print("Predictor interactivo de partidos VCT")
print("=" * 60)

# Crear widgets
team_a_dropdown = Dropdown(
    options=available_teams,
    description='Team A:',
    style={'description_width': '100px'},
    layout={'width': '400px'}
)

team_b_dropdown = Dropdown(
    options=available_teams,
    description='Team B:',
    style={'description_width': '100px'},
    layout={'width': '400px'}
)

predict_button = Button(
    description='Predecir Resultado',
    button_style='success',
    layout={'width': '200px'}
)

output_area = Output()

# Función que se ejecuta al hacer clic
def on_predict_click(b):
    with output_area:
        clear_output()
        
        team_a = team_a_dropdown.value
        team_b = team_b_dropdown.value
        
        if team_a == team_b:
            print("ERROR: Debes seleccionar equipos diferentes")
            return
        
        print(f"\n{team_a} vs {team_b}")
        print("=" * 60)
        print("\nCalculando predicción...\n")
        
        # Hacer predicción
        resultado = predict_match(team_a, team_b)
        
        if "error" in resultado:
            print(resultado["error"])
            return
        
        # Mostrar resultado
        print("=" * 60)
        print("Resultado de la predicción")
        print("=" * 60)
        print(f"\nGanador predicho: {resultado['ganador_predicho']}")
        print(f"\nPROBABILIDADES:")
        print(f"   • {team_a}: {resultado['probabilidad_team_a']:.1f}%")
        print(f"   • {team_b}: {resultado['probabilidad_team_b']:.1f}%")
        print(f"\nConfianza del modelo: {resultado['confianza']:.1f}%")
        
        # Gráfico visual simple
        prob_a = resultado['probabilidad_team_a']
        prob_b = resultado['probabilidad_team_b']
        
        bar_a = '█' * int(prob_a / 2)
        bar_b = '█' * int(prob_b / 2)
        
        print(f"\nVISUALIZACIÓN:")
        print(f"   {team_a[:20]:20} {bar_a} {prob_a:.1f}%")
        print(f"   {team_b[:20]:20} {bar_b} {prob_b:.1f}%")
        print("\n" + "=" * 60)

# Conectar botón con función
predict_button.on_click(on_predict_click)

# Mostrar interfaz
print("\nSelecciona los equipos y haz clic en 'Predecir':\n")

display(VBox([
    HBox([team_a_dropdown]),
    HBox([team_b_dropdown]),
    HBox([predict_button]),
    output_area
]))

Predictor interactivo de partidos VCT

Selecciona los equipos y haz clic en 'Predecir':

